In [2]:
import concurrent.futures
import csv
import requests
from tqdm import tqdm
import json
from concurrent.futures import ThreadPoolExecutor
from queue import Queue



def call_large_model_api(user_prompt, user_input):
    url = 'https://u21829-bb5a-4d812fe5.westc.gpuhub.com:8443/v1/chat/completions'  # 请替换为实际的API URL
    headers = {
        'Content-Type': 'application/json',
        # 请替换为实际的API密钥
    }
    data = {
        "model": "llama3",
        "stream": False,
        "temperature": 0.01,
        "top_p": 0.8,
        "do_sample": True,
        "messages": [
            {
                "role": "system",
                "content": user_prompt
            },
            {
                "role": "user",
                "content": user_input
            }
        ]
    }
    
    response = requests.post(url, json=data, headers=headers)
    
    return response





# 处理每一行的函数
def process_row(row, result_queue):
    try:
        json_data = {}
        # 调用大模型API提取技术栈
        prompt = f'我会给你一个json,例如："职位名称_1":["数量": num,"技能":["技能1": skill_1, "重要性": times]]。然后给你一个 职位名称 + 职位描述 + 技能栈 。你需要把这个增添到我的json中，如果json中已经有相似的职位，则把 职位名称对应的数量+1，并且更新该职位中的技能，没有的置为1，有的就+1。最后只给我返回我需要的json ,不得生成其他文字。\n职位名称：{row[0]}, 职位描述: {row[12]},技能栈：{row[18]}'
        json_string = json.dumps(json_data)
        
        response = call_large_model_api(prompt, json_string)
        skill = response.json()
        result_queue.put([skill["choices"][0]["message"]["content"]])
        
    except Exception as e:
        print(f"Error processing row: {e}")

# 主函数
def main():
    # 初始化队列
    result_queue = Queue()
    info = []

    # 打开输入文件
    with open('Boss直聘/Boss直聘_skills.csv', 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # 跳过标题行

        # 使用线程池并行处理数据
        with ThreadPoolExecutor(max_workers=80) as executor:
            # 提交所有任务
            futures = [executor.submit(process_row, row, result_queue) for row in reader]

            # 遍历future对象，获取结果
            for future in tqdm(as_completed(futures), total=len(futures), desc="Processing rows", unit="row"):
                # 等待任务完成并获取结果
                info.append(result_queue.get())
                
    print(info)
# 运行主函数
if __name__ == "__main__":
    main()


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
{'detail': [{'type': 'string_type', 'loc': ['body', 'messages', 1, 'content', 'str'], 'msg': 'Input should be a valid string', 'input': {'data': {'position_name': '后端开发工程师', 'job_description': '岗位职责：1. 负责完成产品/项目的软件需求文档编制、代码编写和功能测试工作；2. 负责维护产品/项目在使用过程中出现的问题；3. 负责给客户提供操作培训、实施；4. 负责整理归档设计资料。任职要求：1、24-28岁以内，本科以上学历计算机相关专业，2年以上工作经验，具有能源数字物联网平台相关模块的开发经验优先考虑；2、精通.NetCore , ASP.NET Core，net8开发，精通CS与BS架构开发；3. 熟悉多线程、异步编程等技术4、熟悉Linux环境下.net core运行的相关配置;5、熟悉Vue框架，了解Restful Api接口规范；6、至少精通一种数据库操作，例如SQL Server、Mysql；熟悉ORM,SqlSugar技术；具有TDengine数据经验优先考虑。7、英语四级以上；8、善于沟通和思考，富有责任心，拥有良好的团队合作精神。', 'skills_stack': '\n任职资格：\n1. 24-28岁以内，本科以上学历计算机相关专业，2年以上工作经验，具有能源数字物联网平台相关模块的开发经验优先考虑；\n2. 精通.NetCore, ASP.NET Core, net8开发，精通CS与BS架构开发；\n3. 熟悉多线程、异步编程等技术；\n4. 熟悉Linux环境下.net core运行的相关配置；\n5. 熟悉Vue框架，了解Restful Api接口规范；\n6. 至少精通一种数据库操作，例如SQL Server、Mysql；熟悉ORM,SqlSugar技术；具有TDengine数据经验优先考虑；\n7. 英语四级以上；\n8. 善于沟通和思考，富有责任心，拥有良好的团队合作精神。'}, 'json': {}}}, {'type': 'list_type', 'loc': ['body', 'messages', 1, 'content', 'list[MultimodalInputItem]'], 'msg': 'Input should be a valid list', 'input': {'data': {'position_name': '后端开发工程师', 'job_description': '岗位职责：1. 负责完成产品/项目的软件需求文档编制、代码编写和功能测试工作；2. 负责维护产品/项目在使用过程中出现的问题；3. 负责给客户提供操作培训、实施；4. 负责整理归档设计资料。任职要求：1、24-28岁以内，本科以上学历计算机相关专业，2年以上工作经验，具有能源数字物联网平台相关模块的开发经验优先考虑；2、精通.NetCore , ASP.NET Core，net8开发，精通CS与BS架构开发；3. 熟悉多线程、异步编程等技术4、熟悉Linux环境下.net core运行的相关配置;5、熟悉Vue框架，了解Restful Api接口规范；6、至少精通一种数据库操作，例如SQL Server、Mysql；熟悉ORM,SqlSugar技术；具有TDengine数据经验优先考虑。7、英语四级以上；8、善于沟通和思考，富有责任心，拥有良好的团队合作精神。', 'skills_stack': '\n任职资格：\n1. 24-28岁以内，本科以上学历计算机相关专业，2年以上工作经验，具有能源数字物联网平台相关模块的开发经验优先考虑；\n2. 精通.NetCore, ASP.NET Core, net8开发，精通CS与BS架构开发；\n3. 熟悉多线程、异步编程等技术；\n4. 熟悉Linux环境下.net core运行的相关配置；\n5. 熟悉Vue框架，了解Restful Api接口规范；\n6. 至少精通一种数据库操作，例如SQL Server、Mysql；熟悉ORM,SqlSugar技术；具有TDengine数据经验优先考虑；\n7. 英语四级以上；\n8. 善于沟通和思考，富有责任心，拥有良好的团队合作精神。'}, 'json': {}}}]}
